# Approximators

> Models that approximate a function

In [ ]:
#| default_exp RL_approximators

In [ ]:
import logging
logging_level = logging.DEBUG

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from abc import ABC, abstractmethod
from typing import Union, Tuple
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import time

In [ ]:
#| export
class BaseApproximator(nn.Module):

    """ Some basic functions for approximators """

    def __init__(self):
        super().__init__()

    def build_MLP(  self,
                    input_size: int,
                    output_size: int,
                    hidden_layers: list,
                    activation: str = "relu",
                    drop_prob: float = 0.0,
                    batch_norm: bool = False,
                    final_activation: str = "identity",
                    init_method: str = "xavier_uniform" # Parameter for initialization
                  ):

        """ Builds a multi-layer perceptron (MLP) """

        HiddenActivation = self.select_activation(activation)
        FinalActivation = self.select_activation(final_activation)

        layers = []

        # Hidden layers
        last_size = input_size
        for num_neurons in hidden_layers:
            
            linear_layer = nn.Linear(last_size, num_neurons)
            self.init_weights(linear_layer, init_method, activation)
            layers.append(linear_layer)
        
            layers.append(HiddenActivation())
            layers.append(nn.Dropout(p=drop_prob))
            if batch_norm:
                layers.append(nn.BatchNorm1d(num_neurons))
            last_size = num_neurons
        
        # Output layer
        output_layer = nn.Linear(last_size, output_size)
        self.init_weights(output_layer, init_method, final_activation)
        layers.append(output_layer)
        layers.append(FinalActivation())

        # Combine layers
        model = nn.Sequential(*layers)

        return model

    def init_weights(self, layer, init_method, activation):
        """ Initialize the weights of a layer """
        init_method_function = self.select_init_method(init_method)
        
        # Check if initialization method requires gain
        if init_method in ["xavier_uniform", "xavier_normal"]:
            if activation == "identity":
                activation = "linear"
            gain = nn.init.calculate_gain(activation)
            init_method_function(layer.weight, gain=gain)
        else:
            init_method_function(layer.weight)

        # print layer weights

    @staticmethod
    def select_init_method(init_method):
        """ Select the initialization method based on input string """
        init_method = init_method.lower()
        if init_method in ["xavier_uniform", "xavier"]:
            return nn.init.xavier_uniform_
        elif init_method in ["xavier_normal", "xaviernorm"]:
            return nn.init.xavier_normal_
        elif init_method in ["he_uniform", "kaiming_uniform"]:
            return nn.init.kaiming_uniform_
        elif init_method in ["he_normal", "kaiming_normal"]:
            return nn.init.kaiming_normal_
        elif init_method in ["normal", "gaussian"]:
            return nn.init.normal_
        elif init_method == "uniform":
            return nn.init.uniform_
        else:
            raise ValueError("Initialization method not recognized")

    @staticmethod
    def select_activation(activation):
        """ Select the activation function based on input string """
        activation = activation.lower()  # Convert input to lowercase for consistency
        if activation == "relu":
            return nn.ReLU
        elif activation == "sigmoid":
            return nn.Sigmoid
        elif activation == "tanh":
            return nn.Tanh
        elif activation == "elu":
            return nn.ELU
        elif activation == "leakyrelu":
            return nn.LeakyReLU
        elif activation == "identity":
            return nn.Identity
        else:
            raise ValueError(f"Activation function {activation} not recognized")

    def forward(self, x):
        """ Forward pass through the network - overwrite this if necessary """
        return self.model(x)

In [ ]:
#| export
class MLPStateAction(BaseApproximator):

    """Multilayer perceptron model for critic networks that take
    both states and actions as inputs to output the q-value"""

    def __init__(self,
                    input_shape: Tuple, # number of features
                    output_shape: Tuple, # number of outputs/actions
                    hidden_layers: list, # list of number of neurons in each hidden layer
                    activation: str = "relu",
                    drop_prob: float = 0.0, # dropout probability
                    batch_norm: bool = False, # whether to apply batch normalization
                    final_activation: str = "identity", # whether to apply ReLU activation to the output
                    init_method: str = "xavier_uniform",  # Parameter for initialization
                    use_cuda: bool = False, # handled by mushroomRL, not used here
                    dropout: bool = False # legacy parameter to ensure compatibility, use drop_prob instead
                    ):

        super().__init__()
        
        self.model = self.build_MLP(    input_shape[0],
                                        output_shape[0],
                                        hidden_layers,
                                        activation, 
                                        drop_prob,
                                        batch_norm,
                                        final_activation,
                                        init_method)

    def forward(self, state, action):


        state_action = torch.cat([state.float(), action.float()], dim=1)

        q = self.model(state_action)

        # TODO: check if squeeze is necessary
        # return q
        return torch.squeeze(q)

In [ ]:
#| export
class MLPState(BaseApproximator):

    """Multilayer perceptron model for critic networks that take
    both states and actions as inputs to output the q-value"""

    def __init__(self,
                    input_shape: Tuple, # number of features
                    output_shape: Tuple, # number of outputs/actions
                    hidden_layers: list, # list of number of neurons in each hidden layer
                    activation: str = "relu",
                    drop_prob: float = 0.0, # dropout probability
                    batch_norm: bool = False, # whether to apply batch normalization
                    final_activation: str = "identity", # whether to apply ReLU activation to the output
                    init_method: str = "xavier_uniform",  # Parameter for initialization
                    use_cuda: bool = False, # handled by mushroomRL, not used here
                    dropout: bool = False # legacy parameter to ensure compatibility, use drop_prob instead
                    ):

        super().__init__()
        
        self.model = self.build_MLP(    input_shape[0],
                                        output_shape[0],
                                        hidden_layers,
                                        activation, 
                                        drop_prob,
                                        batch_norm,
                                        final_activation,
                                        init_method)

    def forward(self, state):

        state = state.float()

        q = self.model(state)

        # TODO: check if squeeze is necessary
        # return q.squeeze()
        return q

In [ ]:
#| export
class MLPActor(BaseApproximator):

    """Multilayer perceptron model for critic networks that take
    both states and actions as inputs to output the q-value"""

    def __init__(self,
                    input_shape: Tuple, # number of features
                    output_shape: Tuple, # number of outputs/actions
                    hidden_layers: list, # list of number of neurons in each hidden layer
                    activation: str = "relu",
                    drop_prob: float = 0.0, # dropout probability
                    batch_norm: bool = False, # whether to apply batch normalization
                    final_activation: str = "identity", # whether to apply ReLU activation to the output
                    init_method: str = "xavier_uniform",  # Parameter for initialization
                    use_cuda: bool = False,
                    dropout: bool = False, # legacy parameter to ensure compatibility, use drop_prob instead
                    **kwargs
                    ): 

        super().__init__()
        
        self.model = self.build_MLP(    input_shape[0],
                                        output_shape[0],
                                        hidden_layers,
                                        activation, 
                                        drop_prob,
                                        batch_norm,
                                        final_activation,
                                        init_method)

    def forward(self, state):

        state = state.float()

        a = self.model(state)

        return a

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()